### Fetch Files from GDrive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# drive.flush_and_unmount()

### Unrar Dataset

In [ ]:
# %pip install unrar
# %unrar x "datasets.rar"

### Download the Results (For GDrive)

In [ ]:
# import shutil
# from google.colab import files

# shutil.make_archive('train_results', 'zip', '/content/runs/detect/teknofest_train/teknofest')
# files.download('train_results.zip')

### Install Dependencies

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.4.14  Python-3.11.9 torch-2.10.0+cpu CPU (11th Gen Intel Core i5-11400F @ 2.60GHz)
Setup complete  (12 CPUs, 15.8 GB RAM, 412.3/464.3 GB disk)


### Initialize Constants

In [ ]:
# Train parameters
# epochs: 50-100 is ideal for the competition but start with 5 for testing.
# imgsz: VisDrone images are large. 640 is standard but 1024 may be needed for small objects (if GPU memory allows).
# batch: Adjust based on GPU memory. Lower if you get errors (e.g., 8, 4, 2).
# device: 0 (uses NVIDIA GPU), cpu (very slow)
# n = nano (the fastest), s = small, m = medium, b = base, l = large, x = extra large

QUIT_KEY = 'q'
DATA_PATH = 'datasets/VisDrone/data.yaml'
EPOCHS = 10
IMAGE_SIZE = 640
BATCH = 8
DEVICE = 0 #"cpu" for non-nvidia gpu's
PROJECT_NAME = 'teknofest_egitim'
NAME = 'teknofest'

"""
box_loss: How well the model is drawing the box. As this number goes down, it means the model is finding the object's location better.

cls_loss (Sınıf Kaybı): Modelin nesneyi ne kadar doğru tanıdığı (Araba mı, kamyon mu?). Bu sayı düştükçe model artık arabayı kamyonla karıştırmaz.

dfl_loss: How well the model is adjusting the box boundaries. As this number goes down, it means the model is better at fine-tuning the box edges to fit the object.

Instances: The number of objects detected in the image. As this number goes up, it means the model is finding more objects.

Size: The size of the model. Larger models (like 'l' or 'x') can capture more details but require more computational resources, while smaller models (like 'n' or 's') are faster but may miss finer details.

"""

# https://docs.ultralytics.com/models/yolov10/#comparisons

# https://www.kaggle.com/datasets/shisuiotsutsuki/visdrone2019-det?resource=download

### Train the Model

In [ ]:

# from ultralytics import YOLO
# MODEL = "yolov10b.pt" # ("yolov10n.pt")  havier but more accurate, "yolov10m.pt" orta yol, "yolov10s.pt" hızlı ama daha az doğru
# model = YOLO(MODEL)

# results = model.train(
#     data=DATA_PATH,
#     epochs=EPOCHS,
#     imgsz=IMAGE_SIZE,
#     batch=BATCH,
#     device=DEVICE,
#     project=PROJECT_NAME,
#     name=NAME
# )

### Validation

In [ ]:
from ultralytics import YOLO
model = YOLO('best.pt') 

results = model.val(data='datasets/VisDrone/data.yaml')

print(f"Precision Mean (mAP50): {results.box.map50:.4f}")
print(f"Recall Mean (mAR50): {results.box.mar50:.4f}")
print(f"General Sensitivity (mAP50-95): {results.box.map:.4f}")

Ultralytics 8.4.14  Python-3.11.9 torch-2.10.0+cpu CPU (11th Gen Intel Core i5-11400F @ 2.60GHz)
YOLOv10b summary (fused): 142 layers, 19,011,822 parameters, 0 gradients, 91.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 464.6127.7 MB/s, size: 254.9 KB)
val: Scanning C:\Users\PC\Desktop\aeromind\yolo\datasets\VisDrone\val\labels... 548 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 548/548 2.0Kit/s 0.3s<0.2s
val: New cache created: C:\Users\PC\Desktop\aeromind\yolo\datasets\VisDrone\val\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 3.2s/it 1:523.3sss
                   all        548      38759      0.509      0.394      0.409      0.246
            pedestrian        520       8844      0.498      0.449      0.455      0.209
                people        482       5125      0.578      0.297       0.36       0.14
               bicycle        364       1287      0.231      0.198      0.138   

### Image Test

In [ ]:
from ultralytics import YOLO
import cv2
MODEL = "best.pt"
model = YOLO(MODEL)

source_img = "test_image.jpg" 
results = model.predict(source=source_img, save=True, show=True, conf=0.25)

print("Test is completed! You can check the 'runs/detect/predict' folder for results.")

cv2.waitKey(0)
cv2.destroyAllWindows()

### Video / Cam Test

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("best.pt")
video_path = "drone_video.mp4" 
cap = cv2.VideoCapture(video_path) #0 for webcam

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

print(f"Press '{QUIT_KEY}' to exit.")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # stream=true
    # conf=0.5: we only show detections with 50% confidence or higher
    # verbose=False: To avoid cluttering the terminal with prediction details
    results = model.predict(frame, conf=0.5, verbose=False)

    # Display the results on the frame
    annotated_frame = results[0].plot()

    # Show the frame with detections
    cv2.imshow("YOLOv10 Object Detection", annotated_frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord(QUIT_KEY):
        break

cap.release()
cv2.destroyAllWindows()